# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [27]:
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

resp = requests.get(URL, headers=HEADERS)
html = resp.text

Use Beautiful Soup to parse this string into a tree called `soup`

In [28]:
soup = BeautifulSoup(html, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [29]:
#len(soup.find_all("table"))

We can use attributes like `class=` and `style=` to narrow down the list.

In [30]:
#print("All tables:", len(soup.find_all("table")))

#print("wikitable tables:", len(soup.select("table.wikitable")))

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [41]:
def list_wikitable_headers(soup):
    tables = soup.select("table.wikitable")
    print("wikitable count:", len(tables))
    def first_row_cells(tbl):
        tr0 = tbl.find("tr")
        return [] if not tr0 else [c.get_text(" ", strip=True) for c in tr0.find_all(["th","td"])]
    for i, t in enumerate(tables):
        hdr = first_row_cells(t)
        print(f"\n[{i}] {' | '.join(hdr)}")
    return soup.select("table.wikitable")

candidates = list_wikitable_headers(soup)

wikitable count: 7

[0]  | State capital

[1] City | ST | 2024 estimate | 2020 census | Change | 2020 land area | 2020 density | Location

[2] Population | Number of municipal governments

[3] State | Number of listed cities

[4] Municipio | 2024 estimate | 2020 census | Change | 2020 land area | 2020 density | Location

[5] Census- designated place | ST | 2020 census | 2010 census | Change | 2020 land area | 2020 density | Location

[6] City | ST | 2024 estimate | Peak population | % decline from peak | Peak year | 


**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [43]:
table = candidates[1]

def norm(s):
    s = s.replace("\xa0", " ")
    s = re.sub(r"\[[^\]]*\]", "", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip().casefold()

header_cells = [th.get_text(" ", strip=True) for th in table.find("tr").find_all(["th","td"])]
header_norm  = [norm(h) for h in header_cells]

def find_col(patterns):
    for i, name in enumerate(header_norm):
        if any(re.search(p, name) for p in patterns):
            return i
    raise KeyError(f"Missing column; looked for {patterns} in {header_cells}")

city_i  = find_col([r"\bcity\b"])
state_i = find_col([r"\bst\b", r"\bstate\b"])
pop_i   = find_col([r"2024", r"estimate"])
land_i  = find_col([r"land", r"area"])

rows = []
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all(["td","th"])
    if len(tds) <= max(city_i, state_i, pop_i, land_i):
        continue
    city = tds[city_i].get_text(" ", strip=True)
    state = tds[state_i].get_text(" ", strip=True)
    pop = tds[pop_i].get_text(" ", strip=True)
    land = tds[land_i].get_text(" ", strip=True)
    rows.append({
        "city": city,
        "state": state,
        "population_latest": pop,
        "land_area_sq_mi_2020": land
    })

df_cities = pd.DataFrame(rows)
df_cities.shape, df_cities.head(5)

((346, 4),
              city state population_latest land_area_sq_mi_2020
 0  New York [ c ]    NY         8,478,072                300.5
 1     Los Angeles    CA         3,878,704                469.5
 2         Chicago    IL         2,721,308                227.7
 3         Houston    TX         2,390,125                640.4
 4         Phoenix    AZ         1,673,164                518.0)

In [44]:
import re
df_cities["city"] = df_cities["city"].map(lambda s: re.sub(r"\[[^\]]*\]", "", s).strip())

In [45]:
df_cities["population_latest"] = (
    df_cities["population_latest"].str.replace(",", "", regex=False).astype(int)
)
df_cities["land_area_sq_mi_2020"] = pd.to_numeric(
    df_cities["land_area_sq_mi_2020"], errors="coerce"
)

Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [46]:
#pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [ ]:
#pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [ ]:
#df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
#df_cities2

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [50]:
#pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

In [52]:
CAT_URL = "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

resp = requests.get(CAT_URL, headers=HEADERS)
resp.raise_for_status()
soup = BeautifulSoup(resp.content, "html.parser")

In [53]:
course_divs = soup.find_all("div", class_="courseblock")
len(course_divs)

74


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [54]:
rows = []

for div in course_divs:
    title_p = div.find("p", class_="courseblocktitle")
    if not title_p:
        continue

    span_units = title_p.find("span")
    if span_units:
        span_units.extract()

    title = title_p.get_text(" ", strip=True)

    if not (title.startswith("DATA") or title.startswith("STAT")):
        continue

    term_p = div.find("p", class_="noindent")
    term_raw = term_p.get_text(" ", strip=True) if term_p else ""

    term_clean = re.sub(r".*Term Typically Offered:\s*", "", term_raw, flags=re.I)

    rows.append({
        "course": title,
        "term_typically_offered": term_clean
    })

df_courses = pd.DataFrame(rows)
df_courses.shape, df_courses.head(8)

((74, 2),
                                               course term_typically_offered
 0                  DATA 100. Data Science for All I.               F, W, SP
 1            DATA 301. Introduction to Data Science.               F, W, SP
 2         DATA 401. Data Science Process and Ethics.                      F
 3  DATA 402. Mathematical Foundations of Data Sci...                      F
 4        DATA 403. Data Science Projects Laboratory.                      F
 5               DATA 441. Bioinformatics Capstone I.                      W
 6              DATA 442. Bioinformatics Capstone II.                     SP
 7                 DATA 451. Data Science Capstone I.                      W)

In [55]:
print(df_courses["course"].str.startswith(("DATA","STAT")).all())
df_courses.sample(5, random_state=1)

True


,course,term_typically_offered
19,STAT 302. Statistics II.,"W, SP"
55,STAT 485. Cooperative Education Experience.,"F, W, SP"
10,STAT 130. Introduction to Statistical Reasoning.,"F, W, SP"
70,STAT 566. Graduate Consulting Practicum.,SP
54,STAT 470. Selected Advanced Topics.,TBD


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag